Trying to emulate what lich.rb::XMLParser does to get a handle on how it works

In [6]:
import pathlib

log = pathlib.Path.cwd().parents[2] / "revenant_game.log"
log.exists()

True

In [7]:
with open(log) as infile:
    lines = infile.readlines()

In [15]:
lines[100:105]

["<compDef id='exp Outdoorsmanship'></compDef>\n",
 "<compDef id='exp Skinning'></compDef>\n",
 "<compDef id='exp Forging'></compDef>\n",
 "<compDef id='exp Engineering'></compDef>\n",
 "<compDef id='exp Outfitting'></compDef>\n"]

In [12]:
server_time = None
last_tag = ""
last_id = ""
def reset():
    current_stream = ""
    current_style = ""
    active_tags = []
    active_ids = []


"<playerID id='440984'/>\n"

Unescaping numeric character references

In [16]:
import html

In [32]:
lines[297]

'<compass></compass><prompt time="1626723480">&gt;</prompt>\n'

In [33]:
html.unescape(lines[297])

'<compass></compass><prompt time="1626723480">></prompt>\n'

In [37]:
t = '<prompt time="1626723480">&gt;</prompt>\n'

In [44]:
import re


```
   if str = @buffer.slice!(/^[^<]+/)
      text(str.gsub(/&(lt|gt|quot|apos|amp)/) { @unescape[$1] })
   elsif str = @buffer.slice!(/^<\/[^<]+>/)
      element = /^<\/([^\s>\/]+)/.match(str).captures.first
      tag_end(element)
   elsif str = @buffer.slice!(/^<[^<]+>/)
      element = /^<([^\s>\/]+)/.match(str).captures.first
      attributes = Hash.new
      str.scan(/([A-z][A-z0-9_\-]*)=(["'])(.*?)\2/).each { |attr| attributes[attr[0]] = attr[2] }
      tag_start(element, attributes)
      tag_end(element) if str =~ /\/>$/
   else
      break
   end

In [143]:
active_tags = []
bold = False
player_id = None
game = None

unesc_re = re.compile(r"^[^<]+")
end_re = re.compile(r"^<\/[^<]+>")
end_info_re = re.compile(r"^<\/([^\s>\/]+)")
start_re = re.compile(r"^<[^<]+>")
start_info_re = re.compile(r"^<([^\s>\/]+)")
attr_re = re.compile(r"([A-z][A-z0-9_\-]*)=([\"'])(.*?)\2")


def text(text_string):
    return text_string

def tag_start(name, attributes):
    active_tags.append(name)
    if name == "pushBold":
        self.bold = True
    elif name == "popBold":
        self.bold = False
    elif name == 'playerID':
        print(name)
        print(attributes)
        self.player_id = attributes['id']
    elif name == "settingsInfo":
        if 'instance' in attributes:
            self.game = attributes['instance']


for line in lines:
    m = unesc_re.match(line)
    if m:
        text(html.unescape(line))
        continue
        # Its not a tag
        # print(html.unescape(line))
    m = end_re.match(line)
    if m:
        print(m)
        info_match = end_info_re.match(m.group(0))
        if info_match:
            print(info_match)
        continue
    m = start_re.match(line)
    if m:
        info = start_info_re.match(m.group(0))
        if info:
            element = info.group(1)
            attributes = {}
            for attr in attr_re.finditer(line):
                attributes[attr.group(1)] = attr.group(3)
            tag_start(element, attributes)

playerID
{'id': '440984'}


In [191]:
# TODO: Visitor pattern?

class XMLParser:
    """Parser for Dragonrealms tags, directly translated from lich.rb"""
    def __init__(self):
        self.buffer = ""
        self.active_tags = []
        self.last_tag = None
        self.active_ids = []
        self.last_id = None
        self.bold = False
        self.player_id = None
        self.game = None
        self.current_stream = ""
        self.current_style = ""
        self.prompt = ""

        # TODO: Rename unesc, its actually just looking for anything thats not a tag. Unescaping is just an addon.
        self.unesc_re = re.compile(r"^[^<]+")
        self.end_re = re.compile(r"^<\/[^<]+>")
        self.end_info_re = re.compile(r"^<\/([^\s>\/]+)")
        self.start_re = re.compile(r"^<[^<]+>")
        self.start_info_re = re.compile(r"^<([^\s>\/]+)")
        self.attr_re = re.compile(r"([A-z][A-z0-9_\-]*)=([\"'])(.*?)\2")

    def text(self, text_string):
        if self.active_tags and self.active_tags[-1] == 'prompt':
            self.prompt = text_string
            print("prompt: ", text_string)
        return text_string

    def parse(self, line):
        self.buffer += line
        m = self.unesc_re.match(line)
        if m:
            print("line: ", line)
            self.buffer = self.unesc_re.sub(self.buffer, "")
            line = self.unesc_re.sub(self.buffer, "")
            # line = html.unescape(line)
            self.text(html.unescape(line))
            if line:
                return line
        m = self.end_re.match(line)
        if m:
            print("end_re match: ", m)
            info_match = self.end_info_re.match(m.group(0))
            if info_match:
                print(info_match)
            return line
        m = start_re.match(line)
        if m:
            info = start_info_re.match(m.group(0))
            if info:
                element = info.group(1)
                attributes = {}
                for attr in attr_re.finditer(line):
                    attributes[attr.group(1)] = attr.group(3)
                self.tag_start(element, attributes)
                self.tag_end(element)
            return line
        return line
        
    def tag_start(self, name, attributes):
        self.active_tags.append(name)
        if 'id' in attributes:
            self.active_ids.append(attributes['id'])

        if name == "pushBold":
            self.bold = True
        elif name == "popBold":
            self.bold = False
        elif name == 'playerID':
            self.player_id = attributes['id']
        elif name == "settingsInfo":
            if 'instance' in attributes:
                self.game = attributes['instance']

    def tag_end(self, name):
        if self.active_tags:
            self.last_tag = self.active_tags.pop()
        if self.active_ids:
            self.last_id = self.active_ids.pop()


    def reset(self):
        self.current_stream = ""
        self.current_style = ""
        self.active_tags = []
        self.active_ids = []

In [192]:
processed_lines = []
parser = XMLParser()
for line in lines:
    proc_line = parser.parse(line)
    processed_lines.append(proc_line)

line:  Please wait for connection to game server.

line:  

line:  Welcome to DragonRealms (R) v2.00

line:  Copyright 2021 Simutronics Corp.

line:  All Rights Reserved

line:    some polished thin-edged zills with silvered esoteric filigree

line:    a floppy boar-hide hat

line:    a lumium ring helm

line:    a kyanite gwethdesuan

line:    a dull serpent earcuff

line:    a dull serpent earcuff

line:    a lumium ring mask

line:    a gaethzen sphere

line:    a dark watersilk bag bearing a detailed cambrinth medallion

line:    a soft white thigh quiver of snow leopard fur artfully wrapped with silver thread

line:    a narrow gold bracer engraved with the seal of the Lunar Accord

line:    a crystal-inset oaken staff surmounted with a lumpy spleen

line:    a heavy silk duffel bag

line:    a light cherry forester's crossbow with a slender stock

line:    a large hunting pack crafted from wyvern hide

line:    an elegant charcoal black linen shirt embellished with monogrammed bl

In [193]:
# potential unit tests
assert parser.player_id == '440984'
parser.active_ids
parser.prompt

''

In [188]:
processed_lines[:100]

['Please wait for connection to game server.\n',
 "<playerID id='440984'/>\n",
 '<mode id="GAME"/><settingsInfo  client="1.0.1.26" major="652" crc=\'1714388481\' instance=\'DR\'/>\n',
 '\n',
 'Welcome to DragonRealms (R) v2.00\n',
 'Copyright 2021 Simutronics Corp.\n',
 'All Rights Reserved\n',
 '<mode id="GAME"/>\n',
 '<app char="Crannach" game="DR" title="[DR: Crannach] StormFront"/>\n',
 '<streamWindow id="main" title="Story" location="center" target="drop" resident="true"/>\n',
 "<streamWindow id='inv' title='My Inventory' target='wear' ifClosed='' resident='true'/><clearStream id='inv' ifClosed=''/><pushStream id='inv'/>Your worn items are:\n",
 '  some polished thin-edged zills with silvered esoteric filigree\n',
 '  a floppy boar-hide hat\n',
 '  a lumium ring helm\n',
 '  a kyanite gwethdesuan\n',
 '  a dull serpent earcuff\n',
 '  a dull serpent earcuff\n',
 '  a lumium ring mask\n',
 '  a gaethzen sphere\n',
 '  a dark watersilk bag bearing a detailed cambrinth medallion\n',


In [111]:
m = start_re.match(lines[1])
start_info = start_info_re.match(m.group(0))
attr_re.search(lines[1])

<re.Match object; span=(10, 21), match="id='440984'">